In [8]:
%%time
from twython import Twython
from twython import TwythonError, TwythonRateLimitError, TwythonAuthError
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
import time
import datetime
#import csv
#import nltk
#import nltk.tag.stanford as st
import ner

CPU times: user 25 µs, sys: 8 µs, total: 33 µs
Wall time: 37 µs


In [9]:
CONSUMER_KEY = 'VEW2a7ttqT7rUrfi88ql02rpd'
CONSUMER_SECRET ='susbqqPl1WU3C8d4jGgxaCUBebmYuPRXljwQr4N858FXyVCMFj'
OAUTH_TOKEN = '27459655-dGoeUZP6x92W7C97ZBp2kuZWnnFBDBfRmJgtD9fwS'
OAUTH_TOKEN_SECRET = 'nP7CfdiiIhvjzUlUg38qgVoR5nWjWqpv8NaIcxpMzY8Yp'

auth = twitter.oauth.OAuth(OAUTH_TOKEN, OAUTH_TOKEN_SECRET,
                           CONSUMER_KEY, CONSUMER_SECRET)
api = Twython(CONSUMER_KEY, CONSUMER_SECRET, OAUTH_TOKEN, OAUTH_TOKEN_SECRET)

In [10]:
tagger = ner.SocketNER(host='localhost', port=8080)
tagger.get_entities("University of California is located near the house of jsparrow")

{u'ORGANIZATION': [u'University of California']}

In [42]:
usmall = "https://twitter.com/search?q=audl%20since%3A2015-01-01%20until%3A2015-04-02&src=typd"
driver = webdriver.Chrome()
driver.set_window_size(500, 500)
driver.get(usmall)
tweet_elements = driver.find_elements_by_css_selector(full_tweet_css)
for elt in tweet_elements:
    try:
        t_text = elt.find_element_by_css_selector(tweet_text_css).text
    except NoSuchElementException:
        print "WARNING: No Text Element around: ", index
        t_text = "----NONE----"
        #t_text = elt.get_attribute('innerHTML')
    try:
        t_unixtime = (int(elt.find_element_by_css_selector(timestamp_css)
                  .find_element_by_css_selector("span")
                  .get_attribute("data-time")))
    except NoSuchElementException:
        print "WARNING: No Time Element around: ", index
        #t_unixtime = elt.get_attribute('innerHTML')
        # defaults to 1/1/2016 (impossible, so it will show up as a red flag in the data)
        t_unixtime = 1451606400
    # get any quoted tweets within the tweet
    print t_text[0:20], t_unixtime
driver.quit()

@DYMongoose @DiscAce 1427929402
My Ultimate Discover 1427914527
Pro #ultimate coming 1427914511
"@atlantahustle: AUD 1427910676
BREAKING @theAUDL NE 1427910313
Finishing up a great 1427907595
Lil Wayne, TI, and 2 1427905152
2Chainz now is a par 1427904945
Get your tickets tod 1427903734
Lil Wayne, 2 Chainz, 1427902434
This season @EvanLep 1427897997
AUDL announces new A 1427895594
昨年末、真駒内で行われた「Year-En 1427886583
Flame Throwers AUDL  1427871556
Hey @a_derbs we owe  1427857864
@EdmontonUlti and Ju 1427852294
A little hotbox afte 1427844930
“@theAUDL: The 2014  1427813282


In [52]:
full_tweet_css = ".js-stream-item.stream-item.stream-item.expanding-stream-item"
tweet_text_css = ".TweetTextSize.js-tweet-text.tweet-text"
timestamp_css = '.tweet-timestamp.js-permalink.js-nav.js-tooltip'
quote_tweet_css = ".QuoteTweet-text.tweet-text.u-dir"
rt_css = '.ProfileTweet-action--retweet.u-hiddenVisually'
fav_css = ".ProfileTweet-action--favorite.u-hiddenVisually"
action_css = '.ProfileTweet-actionCount'

def get_tweets(query_str, min_unixtime):
    tweets = []
    driver = webdriver.Chrome()
    driver.set_window_size(500, 500)
    pt2 = "%20since%3A2015-01-01%20until%3A2015-04-05&src=typd"
    url_to_get = "https://twitter.com/search?q=" + query_str + pt2
    print "URL TO GET: ", url_to_get
    #url_to_get = "https://twitter.com/search?q=" + query_str
    driver.get(url_to_get)

    while True:
        index = len(tweets)
        tweets_whole = driver.find_elements_by_css_selector(full_tweet_css)[index:]
        for elt in tweets_whole:
            # get tweet text
            try:
                t_text = elt.find_element_by_css_selector(tweet_text_css).text
            except NoSuchElementException:
                print "WARNING: No Text Element around: ", index
                t_text = elt.get_attribute('innerHTML')
            # get tweet timestamp
            try:
                t_unixtime = (int(elt.find_element_by_css_selector(timestamp_css)
                          .find_element_by_css_selector("span")
                          .get_attribute("data-time")))
            except NoSuchElementException:
                print "WARNING: No Time Element around: ", index
                #t_unixtime = elt.get_attribute('innerHTML')
                # defaults to 1/1/2016 (impossible, so it will show up as a red flag in the data)
                t_unixtime = 1451606400
            # get any quoted tweets within the tweet
            try:
                t_quote_text = elt.find_element_by_css_selector('.QuoteTweet-text.tweet-text.u-dir').text
                quote_tup = (t_quote_text, t_unixtime, 0)
                tweets.append(quote_tup)
            except NoSuchElementException:
                pass
            # get sum of RTs and favorites
            try:
                rts = (elt
                       .find_element_by_css_selector(rt_css)
                       .find_element_by_css_selector(action_css)
                       .get_attribute("data-tweet-stat-count"))
                favs = (elt
                       .find_element_by_css_selector(fav_css)
                       .find_element_by_css_selector(action_css)
                       .get_attribute("data-tweet-stat-count"))
                rts_plus_favs = int(rts) + int(favs)
            except NoSuchElementException:
                rts_plus_favs = 0
            tweet_tup = (t_text,t_unixtime,rts_plus_favs)
            tweets.append(tweet_tup)
            print "TWEETS 1!!!"
            print tweets
            print "TWEETS -1!!!!"
            print tweets[-1]
            print "min_unixtime:", min_unixtime
            if int(tweets[-1][1]) < min_unixtime:
                driver.quit()
                return(tweets)
            print "Most Recent Unix Time: ", int(tweets[-1][1])
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(3)
    driver.quit()


In [75]:
def get_tweets(query_str, min_unixtime):
    tweets = []
    driver = webdriver.Chrome()
    driver.set_window_size(500, 500)
    pt2 = "%20until%3A2015-04-05&src=typd"
    url_to_get = "https://twitter.com/search?q=" + query_str + pt2
    print "URL TO GET: ", url_to_get
    #url_to_get = "https://twitter.com/search?q=" + query_str
    driver.get(url_to_get)
    
    while True:
        index = len(tweets)
        tweets_whole = driver.find_elements_by_css_selector(full_tweet_css)[index:]
        for elt in tweets_whole:
            # get tweet text
            try:
                t_text = elt.find_element_by_css_selector(tweet_text_css).text
            except NoSuchElementException:
                print "WARNING: No Text Element around: ", index
                t_text = "ERROR: no text element found"
            # get tweet timestamp
            try:
                t_unixtime = (int(elt.find_element_by_css_selector(timestamp_css)
                          .find_element_by_css_selector("span")
                          .get_attribute("data-time")))
            except NoSuchElementException:
                print "WARNING: No Time Element around: ", index
                #t_unixtime = elt.get_attribute('innerHTML')
                # defaults to 1/1/2016 (impossible, so it will show up as a red flag in the data)
                t_unixtime = 1451606400
            # get any quoted tweets within the tweet
            try:
                t_quote_text = elt.find_element_by_css_selector('.QuoteTweet-text.tweet-text.u-dir').text
                quote_tup = (t_quote_text, t_unixtime, 0)
                tweets.append(quote_tup)
            except NoSuchElementException:
                pass
            # get sum of RTs and favorites
            try:
                rts = (elt
                       .find_element_by_css_selector(rt_css)
                       .find_element_by_css_selector(action_css)
                       .get_attribute("data-tweet-stat-count"))
                favs = (elt
                       .find_element_by_css_selector(fav_css)
                       .find_element_by_css_selector(action_css)
                       .get_attribute("data-tweet-stat-count"))
                rts_plus_favs = int(rts) + int(favs)
            except NoSuchElementException:
                rts_plus_favs = 0
            tweet_tup = (t_text,t_unixtime,rts_plus_favs)
            tweets.append(tweet_tup)
        if int(tweets[-1][1]) < min_unixtime:
            print "FINISHED, WITH LAST TWEET AT: ", tweets[-1][1]
            driver.quit()
            return(tweets)
        print "Most Recent Unix Time: ", int(tweets[-1][1])
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)
    driver.quit()

In [77]:
%%time
#tweets = get_tweets("audl", 1420070400)
tweets_pt2 = get_tweets("audl", 1420070400)

URL TO GET:  https://twitter.com/search?q=audl%20until%3A2015-04-05&src=typd
Most Recent Unix Time:  1427914511
Most Recent Unix Time:  1427719182
Most Recent Unix Time:  1427250695
Most Recent Unix Time:  1426712647
Most Recent Unix Time:  1426091940
Most Recent Unix Time:  1425667206
Most Recent Unix Time:  1425099048
Most Recent Unix Time:  1424711781
Most Recent Unix Time:  1424273459
Most Recent Unix Time:  1423763227
Most Recent Unix Time:  1423248007
Most Recent Unix Time:  1422676426
Most Recent Unix Time:  1422135566
Most Recent Unix Time:  1421871007
Most Recent Unix Time:  1421511131
Most Recent Unix Time:  1421185508
Most Recent Unix Time:  1420930534
Most Recent Unix Time:  1420652578
Most Recent Unix Time:  1420236100
FINISHED, WITH LAST TWEET AT:  1419806391
CPU times: user 4.43 s, sys: 1.24 s, total: 5.67 s
Wall time: 2min 13s


In [80]:
# print len(tweets_pt2)
# print tweets_pt2[-1]

382
(u'Interesting that @PGHThunderbirds will be in the Midwest not the East #powershift RT @theAUDL: 2015 #AUDL ', 1419806391, 1)


In [85]:
# print len(tweets)
# print tweets[0:5]

2575
[(u'#ultimate Sin The Fields: Rochester Dragons, AUDL Advice, & Reader Mail http://ow.ly/38GyMr ', 1449265620, 0), (u"New STF! This time we tried the strategy of not releasing it 10 mins before a global tragedy! (finger's crossed) http://ultiworld.com/2015/12/04/sin-the-fields-rochester-dragons-audl-advice-reader-mail/ \u2026 \u2026", 1449258052, 1), (u'.@SinTheFields: Rochester Dragons, @theAUDL Advice, & Reader Mail: http://ultiworld.com/2015/12/04/sin-the-fields-rochester-dragons-audl-advice-reader-mail/ \u2026', 1449257125, 4), (u"We're making the storyboard for the 2016 #AUDL mag. What do you want to see in it? ", 1449180084, 8), (u'What a pro looks like.\n#AUDL\n#SOTG ', 1449172301, 18)]


In [84]:
#len(tweets)

2575

In [86]:
## Write to file
with open('audl_tweets_pt2.csv','w') as out:
    csv_out=csv.writer(out)
    csv_out.writerow(['tweet_text','tweet_unixtime', 'tweet_popularity'])
    for row in tweets_pt2:
        new_tup = (row[0].encode("utf8"), row[1], row[2])
        csv_out.writerow(new_tup)

In [ ]:
## Extract user mentions & replace them with names
user_dictionary = {}
tweets_processed = 0
def get_mentions_name(tweet_text,twitter_instance):
    users = re.findall(r'@(\w+)', tweet_text)
    for username in users:
        try:
            dummy = user_dictionary[u]
        except KeyError:
            try:
                name = twitter_instance.show_user(screen_name=username)['name']
                user_dictionary[u] = name.title()
            except TwythonError:
                user_dictionary[u] = '@' + u
    return
def replace_username(old_tweetsnames.csve_dict):
    new_tweets = []
    for (t,time,pop) in tweets:
        for user in re.findall(r'@(\w+)', tweet_text)
            nm = name_dict[user]
            t = re.sub(user,nm,t)
        t = re.sub('@','',t)
        nt = (t,time,pop)
        new_tweets.append(nt)
    return(new_tweets)
        
for (txt,_,_) in tweets:
    get_mentions_name(txt, api)
    tweets_processed += 1

processed = replace_username(tweets,user_dictionary)


In [ ]:
## Find names in text
name_dict =  defaultdict(int)
# Run in different terminal before executing code below
#java -mx1000m -cp stanford-ner.jar edu.stanford.nlp.ie.NERServer \
#     -loadClassifier classifiers/english.all.3class.caseless.distsim.crf.ser.gz \
#     -port 8080 -outputFormat inlineXML
tagger = ner.SocketNER(host='localhost', port=8080)
for (twt,unixtime,pop) in processed:
    persons = tagger.get_entities(t)['PERSON']
    for person in persons:
        name_dict[person.lower()] += pop
        
    

In [114]:
# %%time
# driver = webdriver.Chrome()
# driver.set_window_size(500, 500)
# driver.get("https://twitter.com/search?q=audl")
# full_tweet_css = ".js-stream-item.stream-item.stream-item.expanding-stream-item"
# tweet_text_css = ".TweetTextSize.js-tweet-text.tweet-text"
# timestamp_css = '.tweet-timestamp.js-permalink.js-nav.js-tooltip'
# tweets = []
# while True:
#     index = len(tweets)
#     tweets_whole = driver.find_elements_by_css_selector(full_tweet_css)[index:]
#     for elt in tweets_whole:
#         try:
#             t_text = elt.find_element_by_css_selector(tweet_text_css).text
#         except NoSuchElementException:
#             print "WARNING: No Text Element around: ", index
#             t_text = elt.get_attribute('innerHTML')
#         try:
#             t_unixtime = (elt.find_element_by_css_selector(timestamp_css)
#                       .find_element_by_css_selector("span")
#                       .get_attribute("data-time"))
#         except NoSuchElementException:
#             print "WARNING: No Time Element around: ", index
#             t_unixtime = elt.get_attribute('innerHTML')
#         tweet_tup = (t_text,t_unixtime)
#         tweets.append(tweet_tup)
#     if int(tweets[-1][1]) < 1420070400:
#         break
#     print "Most Recent Unix Time: ", int(tweets[-1][1])
#     #tweet_elts = driver.find_elements_by_css_selector('.TweetTextSize.js-tweet-text.tweet-text')[index:]
#     #tweet_elts = [elt.text for elt in tweet_elts]
#     #tweets.extend(tweet_elts)
#     driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#     time.sleep(1)
# driver.quit()



Most Recent Unix Time:  1448385031
Most Recent Unix Time:  1447366180
Most Recent Unix Time:  1447088425
Most Recent Unix Time:  1446671328
Most Recent Unix Time:  1446223772
Most Recent Unix Time:  1445960222
Most Recent Unix Time:  1444764718
Most Recent Unix Time:  1444321082
Most Recent Unix Time:  1444073028
Most Recent Unix Time:  1443545246
Most Recent Unix Time:  1442960912
Most Recent Unix Time:  1442591623
Most Recent Unix Time:  1442355321
Most Recent Unix Time:  1441906563
Most Recent Unix Time:  1441321464
Most Recent Unix Time:  1440990211
Most Recent Unix Time:  1440794931
Most Recent Unix Time:  1440710108
Most Recent Unix Time:  1440497644
Most Recent Unix Time:  1440125444
Most Recent Unix Time:  1439869321
Most Recent Unix Time:  1439576864
Most Recent Unix Time:  1439320327
Most Recent Unix Time:  1439226261
Most Recent Unix Time:  1439163393
Most Recent Unix Time:  1439161416
Most Recent Unix Time:  1439154754
Most Recent Unix Time:  1439141124
Most Recent Unix Tim

In [124]:
tweets[2][0].encode("utf8").replace("\xe2\x80\xa6","...")

'.@theAUDL November News Roundup: http://ultiworld.com/2015/12/03/audl-november-news-roundup/ ...'

In [150]:
# classifier_fp = '/Users/jackiemartinez/stanford-ner-2014-08-27/classifiers/english.all.3class.distsim.crf.ser.gz'
# tagger_fp = '/Users/jackiemartinez/stanford-ner-2014-08-27/stanford-ner.jar'
# tagger = st.StanfordNERTagger(classifier_fp, tagger_fp)
# tagger.tag("Jimmy Mickle winds up for a huck.".split())

[(u'Jimmy', u'PERSON'),
 (u'Mickle', u'PERSON'),
 (u'winds', u'O'),
 (u'up', u'O'),
 (u'for', u'O'),
 (u'a', u'O'),
 (u'huck.', u'O')]

In [69]:
# driver = webdriver.Chrome()
# driver.set_window_size(1120, 550)
# driver.get("https://twitter.com/search?q=audl")
# driver.find_element_by_id('search_form_input_homepage').send_keys("realpython")
# driver.find_element_by_id("search_button_homepage").click()
# print driver.current_url
# tweets_whole = driver.find_elements_by_css_selector('.js-stream-item.stream-item.stream-item.expanding-stream-item')
# t = tweets_whole[12]
# t_text = t.find_element_by_css_selector('.TweetTextSize.js-tweet-text.tweet-text').text
# print t_text
# print t.get_attribute('innerHTML')
#iframe = t.find_element_by_tag_name('iframe')
#driver.switch_to_frame(t.find_element_by_tag_name("iframe"))
#print driver.page_source
# driver.quit()
# for t in tweets_whole:
#     t_card_text = "tcard: "
#     try:
#         t_text = t.find_element_by_css_selector('.TweetTextSize.js-tweet-text.tweet-text').text
#     except NoSuchElementException:
#         t_text = 0
#     try:
#         iframe = driver.find_element_by_css_selector('iframe')
#         driver.switch_to_frame(iframe)
#         t_card_children = driver.find_element_by_css_selector('.TwitterCard-title').find_elements_by_css_selector("*")
#         for elt in t_card_children:
#             try:
#                 t_card_text = t_card_text + str(elt.text)
#             except:
#                 pass
#         driver.switch_to.default_content()
#     except NoSuchElementException:
#         t_card = 0
#     print "reached"
#     print t_text, t_card_text

# for i in range(3):
#     time.sleep(5)
#     driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#     batch = driver.find_elements_by_css_selector('.TweetTextSize.js-tweet-text.tweet-text')[len(tweets):]
#     for elt in batch:
#         tweets.append(elt)
#     print "i"

https://twitter.com/search?q=audl
Defense is key as @keegannorth_23 goes up big against the Indy AlleyCats #2015recap #AUDL http://youtu.be/AsB2oTjEhiU 

    <ol role="presentation" class="expanded-conversation expansion-container js-expansion-container js-navigable-stream">
  <li role="presentation" class="original-tweet-container">
    


  

  <div class="tweet original-tweet js-original-tweet js-stream-tweet js-actionable-tweet js-profile-popup-actionable  



 has-cards








cards-forward








" data-tweet-id="671706879366664192" data-disclosure-type="" data-item-id="671706879366664192" data-permalink-path="/RevolutionAUDL/status/671706879366664192" data-screen-name="RevolutionAUDL" data-name="Cincy Revolution" data-user-id="359768232" data-has-cards="true" data-card2-type="player" data-expanded-footer="<div class=&quot;js-tweet-details-fixer tweet-details-fixer&quot;>



          <div class=&quot;entities-media-container js-media-container&quot; style=&quot;min-height:0px&

In [ ]:
u'Jimmy Mickle Signing With AUDL\u2019s Dallas Roughnecks - Johnny Bravo\u2019s Jimmy Mickle winds up for a huck. Photo

In [28]:
len(tweets)

57

In [29]:
for t in tweets:
    print t.text

Congrats to Reckon player Ryan Balch, the new coach of the AUDL @NSH_NightWatch. Can he wear his Duke hat? 
First of two free #RevCity recruit camps this Sat. 12/5 at 8pm inside Sheakley! Tryouts in January: http://cincinnatirevolution.com/2016-revolution-tryouts/ … #AUDL
Which of these teams is mostly likely to win the 2016 AUDL championship?
@Morgatron8 we are no longer friends. How could you move to the AUDL??!!?
Three champions--one college, one world, one AUDL--have signed w/ new teams. Details in The Tuesday Toss: http://theaudl.com/tuesdaytoss/2015/SigningsShape2016 … @theAUDL
ICYMI: Jimmy Mickle Signing With @theAUDL's Dallas Roughnecks -> http://ultiworld.com/2015/12/01/jimmy-mickle-signing-with-audls-dallas-roughnecks/ …
Jimmy Mickle @livelaughlime has decide to play for Dallas Roughnecks @DRoughnecks in the AUDL @theAUDL next year. https://www.facebook.com/jmickle/posts/10153938209014050 …
Jimmy Mickle Signing With AUDL’s Dallas Roughnecks - Johnny Bravo’s Jimmy Mickle winds

In [93]:
driver.quit()

In [97]:
driver = webdriver.Chrome()
driver.set_window_size(1120, 550)
driver.get("https://twitter.com/search?q=audl")
print "here: ", datetime.datetime.now().time()
tweets_whole = driver.find_element_by_css_selector('.tweet-timestamp.js-permalink.js-nav.js-tooltip')
tweets_whole = tweets_whole.find_element_by_tag_name("span")
print tweets_whole.get_attribute("data-time")
print "here2: ", datetime.datetime.now().time()
driver.quit()


here:  14:12:04.146091
1449168409
here2:  14:12:04.218219
